# The goal is providing a comparative report on MS and GEM geo-data sets to analyse risks in terms of number of buildings and areas covered/ reported

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

import requests
import zipfile
import os
import json
from shapely.geometry import shape
from shapely.geometry import Polygon

from pyproj import CRS

## Step 1. Get the country name from user


In [ ]:
country_name = input("Enter the name of the country: ")


In [ ]:
# creates file path 

file_name = f"{country_name}.geojsonl"
base_dir ="E:/2023/Natcat/"

geojsonl_file_path = os.path.join(base_dir, f"{country_name}.geojsonl")
geojsonl_file_path

## Step 2. Open the file and read its content line by line
The data type is JSON which is more suitable for unstructured data like the one we have in this project

In [ ]:
with open(geojsonl_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

The following line is used to read a JSON Lines (JSONL) file and convert its contents into a list of JSON objects (Python dictionaries). 

In [ ]:
json_data = [json.loads(line) for line in lines]

## Step 3. Convert JSON objects to Pandas df
is useful for turning complex JSON data structures into a more tabular form that can be easily analyzed and manipulated using Pandas.
Obviously the df contains two columns type as being Polygon and Coordinates. In the next steps I delve into the data set and explore it. 

In [ ]:
df = pd.json_normalize(json_data)
df.head(3)

## Step 4. Exploring the df
Shape (size of df), number of unique entries (nunique) and so on

In [ ]:
df.tail(3)

In [ ]:
df.shape

Now checking if duplicates exist in data frame!

The elements in the coordinates column are lists, which are mutable and therefore unhashable. The nunique function tries to use the elements of the column as keys in a dictionary (to count unique elements), but lists cannot be used as dictionary keys due to their unhashable nature. To work around this, I have converted the lists to tuples (which are immutable and hashable) before applying the nunique method.

In [ ]:
def convert_to_tuple(item):
    if isinstance(item, list):
        return tuple(convert_to_tuple(sub_item) for sub_item in item)
    return item
    
df['coordinates_tuple'] = df['coordinates'].apply(convert_to_tuple)

In [ ]:
df['coordinates_tuple'].nunique()

## Step 5. Converting coordinates to Polygons and getting areas
The generated Polygons are stored in a new column called 'geometry' in the data-frame

In [ ]:
# Convert coordinates to Polygon objects
df['geometry'] = df['coordinates'].apply(lambda x: Polygon(x[0]))

In [ ]:
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84
gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
# Function to calculate UTM zone based on longitude
def utm_zone(longitude):
    return int((longitude + 180) / 6) + 1


In [ ]:
# Function to generate UTM CRS for a given longitude
def get_utm_crs(longitude):
    zone = utm_zone(longitude)
    return f"+proj=utm +zone={zone} +datum=WGS84 +units=m +no_defs"


In [ ]:
# Calculate area in square meters
def calculate_area_in_sq_meters(gdf):
    areas = []
    for index, row in gdf.iterrows():
        utm_crs = get_utm_crs(row['geometry'].centroid.x)
        gdf_utm = gdf.to_crs(utm_crs)
        area = gdf_utm.at[index, 'geometry'].area
        areas.append(area)
    return areas

In [ ]:
# Calculate area and add it to the GeoDataFrame
gdf['area_sq_meters'] = calculate_area_in_sq_meters(gdf)

# Display the GeoDataFrame with areas
print(gdf[['geometry', 'area_sq_meters']])

Below Polygon centroids is determined,  coordinates transformed to UTM and areas calculated.
Coordinate Representation:

Geographic coordinates (WGS84) use angles (degrees) to represent positions on a spherical surface.
Projected coordinates (UTM) use Cartesian coordinates (easting and northing) to represent positions on a flat surface.

In [ ]:
gdf[['area_sqm']].head()

In [ ]:
print("Mean:\n", gdf[['area_sqm']].mean())
print("Median:\n", gdf[['area_sqm']].median())
#print("Mode:\n", gdf[['area_sqm']].mode())
print("Variance:\n", gdf[['area_sqm']].var())
print("Standard Deviation:\n", gdf[['area_sqm']].std())
print("Skewness:\n", gdf[['area_sqm']].skew())
print("Kurtosis:\n", gdf[['area_sqm']].kurtosis())

In [ ]:
print(len(gdf[['area_sqm']]))

In [ ]:
import matplotlib.pyplot as plt
# Filter rows where area_sqm < 10000
filtered_gdf = gdf[gdf['area_sqm'] < 1000]

# Plot histogram
plt.hist(filtered_gdf['area_sqm'], bins=20, color='blue', edgecolor='black')
plt.title('Histogram of Area (Sq. Meters)')
plt.xlabel('Area (Sq. Meters)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
gdf[['type', 'coordinates', 'area_sqm']].head()

In [ ]:
import geopandas as gpd

country_abbreviation = input("Enter the country abbreviation (e.g., ECU for Ecuador): ").strip().upper()
# Load the GADM shapefile for Ecuador (adjust the path to where you downloaded the file)
file_path = rf'E:/2023/Natcat/gadm41_{country_abbreviation}_1.shp'
provinces_gdf = gpd.read_file(file_path)
# Ensure both GeoDataFrames have the same CRS
provinces_gdf = provinces_gdf.to_crs(gdf.crs)

In [ ]:
provinces_gdf.head()

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [ ]:
# Perform spatial join using the 'predicate' parameter
gdf_with_provinces = gpd.sjoin(gdf, provinces_gdf, how="left", predicate="within")

In [ ]:
gdf_with_provinces = gdf_with_provinces.rename(columns={'NAME_1': 'province_name'})

In [ ]:
print(gdf_with_provinces[['type', 'coordinates', 'province_name']].head())

In [ ]:
# Count the number of rows (polygons) in each province
province_counts = gdf_with_provinces.groupby('province_name').size().reset_index(name='counts')


# Display the counts
print(province_counts)

In [ ]:
gdf_with_provinces = gpd.sjoin(gdf, provinces_gdf, how="left", predicate="within")

In [ ]:
gem_df = normalize_province_names(gem_df, 'province')
ms_df = normalize_province_names(ms_df, 'province')

In [ ]:
province_counts = gdf_with_provinces.groupby('NAME_1').size().reset_index(name='counts')
province_areas = gdf_with_provinces.groupby('NAME_1')['area_sqm'].sum().reset_index()

merged_df = pd.merge(province_counts, province_areas, on='NAME_1')
merged_df.rename(columns={'NAME_1': 'province_name'}, inplace=True)
merged_df.to_csv('ecu_MS.csv', index=False)


#gdf_with_provinces[['NAME_1', 'area_sqm']]
merged_df

In [ ]:
gdf_with_provinces

In [ ]:
import matplotlib.pyplot as plt
province_counts_sorted = province_counts.sort_values(by='counts', ascending=False)

# Calculate frequencies
total_polygons = province_counts_sorted['counts'].sum()
province_counts_sorted['frequency'] = province_counts_sorted['counts'] / total_polygons

# Calculate cumulative frequencies
province_counts_sorted['cumulative_frequency'] = province_counts_sorted['frequency'].cumsum()

# Find index where cumulative frequency reaches 80%
highlight_index = (province_counts_sorted['cumulative_frequency'] <= 0.8).sum()

# Plot Pareto histogram
plt.figure(figsize=(10, 6))

# Plot all bars
plt.bar(province_counts_sorted['province_name'], province_counts_sorted['frequency'], color='blue')

# Highlight bars contributing to 80% cumulative frequency
plt.bar(province_counts_sorted['province_name'][:highlight_index], 
        province_counts_sorted['frequency'][:highlight_index], 
        color='green')

plt.title('Polygon Count by Province (Descending)')
plt.xlabel('Province')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.grid(True, linestyle='--', alpha=0.5)
plt.gca().set_axisbelow(True)
plt.tight_layout()
plt.show()

In [ ]:
# Merge the counts with the province boundaries GeoDataFrame
provinces_gdf = provinces_gdf.merge(province_counts, left_on='NAME_1', right_on='province_name', how='left')


In [ ]:
provinces_gdf

In [ ]:
# Fill NaN values in 'counts' with 0 (if there are provinces with no polygons)
provinces_gdf['counts'] = provinces_gdf['counts'].fillna(0)

In [ ]:
plt.figure(figsize=(12, 8))
provinces_gdf.plot(column='counts', cmap='viridis', legend=True,
                   legend_kwds={'label': "Number of Polygons",
                                'orientation': "horizontal"})
plt.title('Number of Polygons per Province in Ecuador')
plt.show()

In [ ]:
print(gdf.head())